In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

%matplotlib inline

In [6]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

r = requests.get('https://www.amazon.sg/gp/bestsellers/books/ref=zg_bs_pg_'+str(1)+'?ie=UTF8&pg='+str(1), headers=headers)#, proxies=proxies)
content = r.content
soup = BeautifulSoup(content)
print(soup.prettify())

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-sg">
 <!-- sp:feature:head-start -->
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <!-- sp:end-feature:head-start -->
  <script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
  <!-- sp:feature:cs-optimization -->
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="https://images-fe.ssl-images-amazon.com" rel="dns-prefetch"/>
  <link href="https://m.media-amazon.com" rel="dns-prefetch"/>
  <link href="https://completion.amazon.com" rel="dns-prefetch"/>
  <!-- sp:end-feature:cs-optimization -->
  <script type="text/javascript">
   window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=f

In [25]:
titles = soup.select('div.zg-grid-general-faceout div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y')
print(len(titles))

58


In [26]:
titles

[<div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">The Silva Mind Control Method</div>,
 <div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">Jose Silva</div>,
 <div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">Asians in Charge: How to Earn Your Place at the Leadership Table</div>,
 <div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">Kenneth Chan</div>,
 <div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">The Boy, the Mole, the Fox and the Horse</div>,
 <div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">Charlie Mackesy</div>,
 <div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">The Psychology of Money: Timeless lessons on wealth, greed, and happiness</div>,
 <div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">Morgan Housel</div>,
 <div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">Atomic Habits: An Easy &amp; Proven Way to Build Good Habits &amp; Break Bad Ones</div>,
 <div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">James Clear</div>,
 <div class="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y">

In [13]:
authors = soup.select('div._cDEzb_p13n-sc-css-line-clamp-1_1Fn1y')
print(len(authors))

58


In [18]:
ratings = soup.select('div.a-icon-row span.a-icon-alt')
print(len(ratings))

30


In [19]:
no_of_ratings = soup.select('div.a-icon-row span.a-size-small')
print(len(no_of_ratings))

30


In [20]:
prices = soup.select('span._cDEzb_p13n-sc-price_3mJ9Z')
print(len(prices))

30


In [2]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.sg/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [3]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [4]:
df = pd.read_csv("amazon_products.csv")
df.head()

,Book Name,Author,Rating,Customers_Rated,Price


In [5]:
df.shape

(0, 5)